In [1]:
spark

In [2]:
sc.master

'yarn'

In [3]:
!ls

Youvegottofindwhatyoulove.txt  home	   mnt	   srv
bin			       lib	   opt	   sys
boot			       lib32	   output  tmp
copyright		       lib64	   proc    usr
dev			       libx32	   root    var
etc			       lost+found  run	   yellow_tripdata_2018-10.csv
hadoop			       media	   sbin


In [4]:
!hdfs dfs -mkdir hw3

mkdir: `hw3': File exists


In [5]:
!hdfs dfs -mkdir /hw3/input
!hdfs dfs -mkdir /hw3/output

mkdir: `/hw3/input': File exists
mkdir: `/hw3/output': File exists


In [6]:
!hdfs dfs -put Youvegottofindwhatyoulove.txt /hw3/input/

put: `/hw3/input/Youvegottofindwhatyoulove.txt': File exists


In [7]:
!hadoop fs -cat /hw3/input/Youvegottofindwhatyoulove.txt

I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?

It started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting list, got a call in the middle of the night asking: “We have an unexpected baby boy; do you want him

In [12]:
from pyspark import SparkContext
from pyspark import SparkConf
import time

sparkConf = SparkConf() \
        .setAppName("Q1") \

sc = SparkContext.getOrCreate();
#print sc.master

text_file = sc.textFile("/hw3/input/Youvegottofindwhatyoulove.txt")

# count how many sentences 
sum = text_file.flatMap(lambda x: x.split("."))
total=0
for w in sum.take(sum.count()):
    #print(w)
    if not w:
        #print('no')
        continue;
    else:
        total=total+1
print("Total sentences:"+str(total))

# count times for each word
counts = text_file.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').replace('\'',' ').replace('\"',' ').lower()) \
        .flatMap(lambda x: x.split()) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y:x+y) \
        .map(lambda x:(x[1],x[0])) \
        .sortByKey(False)

print("Word\tTimes\tAverage times per Sentences")
print("----\t-----------\t----------------------")
for a,b in counts.take(30):
    print("{}\t{}\t\t\t{:.2f}".format(a, b , float(a)/float(total)))

counts.saveAsTextFile("/hw3/output/out01")

Total sentences:144
Word	Times	Average times per Sentences
----	-----------	----------------------
96	the			0.67
86	i			0.60
71	to			0.49
67	and			0.47
53	it			0.37
48	was			0.33
46	a			0.32
41	of			0.28
38	that			0.26
35	in			0.24
31	you			0.22
30	my			0.21
28	is			0.19
22	had			0.15
19	out			0.13
19	with			0.13
18	me			0.12
17	have			0.12
17	for			0.12
17	so			0.12
16	your			0.11
16	life			0.11
16	all			0.11
15	as			0.10
15	on			0.10
15	what			0.10
14	college			0.10
14	but			0.10
14	be			0.10
13	from			0.09


In [13]:
# check answer in hadoop 
!hadoop fs -cat /hw3/output/out01/part-00000 | head -30

(96, 'the')
(86, 'i')
(71, 'to')
(67, 'and')
(53, 'it')
(48, 'was')
(46, 'a')
(41, 'of')
(38, 'that')
(35, 'in')
(31, 'you')
(30, 'my')
(28, 'is')
(22, 'had')
(19, 'out')
(19, 'with')
(18, 'me')
(17, 'have')
(17, 'for')
(17, 'so')
(16, 'your')
(16, 'life')
(16, 'all')
(15, 'as')
(15, 'on')
(15, 'what')
(14, 'college')
(14, 'but')
(14, 'be')
(13, 'from')


In [14]:
!hdfs dfs -ls /hw3/output/

Found 1 items
drwxr-xr-x   - root hadoop          0 2022-04-16 12:55 /hw3/output/out01


In [15]:
!hdfs dfs -rm -r /hw3/output/out01

Deleted /hw3/output/out01
